In [129]:
from astropy.io import fits as pyfits
import numpy as np

## Documentação

Links úteis:

[Exemplos de máscaras](https://numpy.org/doc/stable/reference/generated/numpy.ma.masked_where.html)

[Máscara em 2D](https://stackoverflow.com/questions/34860570/mask-2d-numpy-array)



# Máscara para regiões

In [130]:
# dimensões da sua imagem
y=np.arange(0,1620,1)
x=np.arange(0,1345,1)

#intervalo, em pixeis, a ser coberto (veja no ds9) 
xmask = (x > 486) & (x < 496)
ymask = (y > 915) & (y < 923)


print(xmask) # mascara em x
print(ymask) # mascara em y

[False False False ... False False False]
[False False False ... False False False]


In [131]:
#Compondo uma máscara em 2D (apenas os pontos com dois valores "True" serão mascaradas). Estamos criando eixos 
# adicionais para cada máscara e unindo elas
mask = xmask[np.newaxis, :] & ymask[:, np.newaxis]

In [132]:
print(mask)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


### Preparando a máscara para uso 

 Vamos trocar o tipo dos dados para float, em vez de booleano:

In [133]:
#Vamos trocar o tipo dos dados 
mask = mask.astype('float')
mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

E transformar 1 (região de leitura com defeito) para nan. E 0 (região sem defeito) para 1:

In [134]:
count=0
count2=0
for i in range(0,1620):
    for j in range(0,1345):
        if mask[i,j] == 1:
            mask[i,j] = 0.02
            count=count+1
            
        if mask[i,j] == 0:
            mask[i,j] = 1
            count2=count2+1
#print(count, count2)
print(mask)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


### Aplicando a máscara em uma imagem

In [135]:
# imagem a ser testada com a correção
chip1_29=pyfits.open('NGC4501.phot.2 teste2.fits')

In [136]:
chip1_29.info()

Filename: NGC4501.phot.2 teste2.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU    2658   (1345, 1620)   float64   


In [137]:
# matriz de dados da imagem (obs: pyfits.getdata tmb funciona). 
im=chip1_29[0].data
im

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [138]:
# a = mask*0.02

In [139]:
# multiplicando a matriz de dados pela máscara
im2=im*mask

### Salvando em fits (aprendi com Natanael =) )

In [140]:
path='/home/natanael/Área de Trabalho/Iniciação Científica/IC Novembro 2021/Subsample/Análogas/NGC4501 JIC/'
#salvando 
# name=chip1_29[0].header['FILENAME'] #nome original do header, não inclui ".fits"
name='NGC4501.phot.2 final.fits'
hdu = pyfits.PrimaryHDU(im2,chip1_29[0].header)
hdul = pyfits.HDUList([hdu])
hdul.writeto(path+'/'+name, overwrite=True) 